In [ ]:
!pip install openai requests bs4 -q

In [ ]:
!pip install --upgrade openai

### Resources

- https://medium.com/@odhitom09/openai-responses-api-a-comprehensive-guide-ad546132b2ed

In [ ]:
import requests
from openai import OpenAI
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [ ]:
client = OpenAI(api_key="")

## Completions API

In [ ]:
def scrapping_for(url: str):
    headers = {
     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    title = soup.title.string if soup.title else "No title found"
    
    for irrelevant in soup.body(["script", "style", "img", "input"]):
        irrelevant.decompose()
    
    text = soup.body.get_text(separator="\n", strip=True)

    return {
        "title": title,
        "text": text,
    }

def messages_for(website):
    system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

    user_prompt = f"You are looking at a website titled {website["title"]}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website["text"]
  
    
    return [
        {
            "role": "system",
            "content": system_prompt, 
        },
        {
            "role": "user",
            "content": user_prompt,             
        }
        
    ]

In [ ]:
def summarize(url):    
    website = scrapping_for(url)
    
    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://medium.com/@johnidouglasmarangon")

In [ ]:
file1 = upload_file("slide.pdf")

In [ ]:
response = make_question(file1, "Give me the generative AI tools")
display(Markdown(response))

## Examples using OpenAI Response API

#### Text Prompting - Instructions

In [ ]:
response = client.responses.create(
    model="gpt-4o",
    instructions="Talk like a math teacher.",
    input="Using only an addition, how do you add eight 8’s and get the number 1000?",
)

print(response.output_text)

#### Text Prompting - Chain of Command (hierarchical instructions)

In [ ]:
response = client.responses.create(
    model="gpt-4o",
    input=[
        {"role": "system", "content": "Talk like a teacher."},
        {"role": "developer", "content": "Explain the answer in detail using bullet points for each step."},
        {"role": "user", "content": "What is the result of the equation 5.93 * 10-2?"},
    ],
)

print(response.output_text)

#### Web Search

In [ ]:

response = client.responses.create(
    model="gpt-4o-mini",
    input="What are the tarifs between the USA and China?",
    tools=[
        {
            "type" : "web_search_preview"
        }
    ]
)

print(response.output_text)

#### Reasoning

In [ ]:
prompt = """
Write a recursive Python function to solve the Fibonacci problem.
Use memorization.
"""

response = client.responses.create(
    model="o3-mini",
    reasoning={"effort": "medium"},
    input=[{"role": "user", "content": prompt}],
)

print(response.output_text)

#### Structured Output - Whitout Parsing

In [ ]:
import json
from typing import List

response = client.responses.create(
    model="gpt-4o",
    input=[
        {
            "role": "system",
            "content": "Extract email and name from text.",
        },
        {
            "role": "user",
            "content": "Customer Alice Philip and her email alice@alice.com.",
        },
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "person_info",
            "schema": {
                "type": "object",
                "properties": {
                    "name": {"type": "string"},
                    "email": {"type": "string"},
                },
                "required": ["name", "email"],
                "additionalProperties": False,
            },
            "strict": True,
        }
    },
)

event = json.loads(response.output_text)

print(event)

### Structured Output - With Parsing

In [ ]:
from pydantic import BaseModel

class PersonInfo(BaseModel):
    name: str
    email: str


response = client.responses.parse(
    model="gpt-4o",
    instructions="Extract email and name from text.",
    input="Customer Alice Philip and her email alice@alice.com.",
    text_format=PersonInfo,
)

response_model = response.output[0].content[0].parsed

print(type(response_model))
print(response_model.model_dump_json(indent=2))

### Function Calling 

In [ ]:
tools = [
    {
        "type": "function",
        "name": "send_email",
        "description": "Send an email to a given recipient with a subject and message.",
        "parameters": {
            "type": "object",
            "properties": {
                "to": {"type": "string", "description": "The recipient email address."},
                "subject": {"type": "string", "description": "Email subject line."},
                "body": {"type": "string", "description": "Body of the email message."},
            },
            "required": ["to", "subject", "body"],
            "additionalProperties": False,
        },
    }
]

response = client.responses.create(
    model="gpt-4o",
    input="Can you send an email to ilan@example.com and katia@example.com saying hi?",
    tools=tools,
)

print(response.output)
print(response.output[0].model_dump_json(indent=2))
print(response.output[1].model_dump_json(indent=2))

#### File Search

In [ ]:
uploaded_file = client.files.create(
        file=open(filename, "rb"),
        purpose="user_data",
)

response = client.responses.create(
    model="gpt-4o",
    input=[
        {
            "role": "user",
            "content": [
                {
                    "type": "input_file",
                    "file_id": uploaded_file.id,
                },
                {
                    "type": "input_text",
                    "text": question,
                },
            ]
        }
    ]
)

pritn(response.output_text)

## Hands-on 

In [ ]:
from IPython.display import Markdown, display, update_display
from openai import OpenAI

question = """
Please explain what this code does and why:

```
yield from {book.get("author") for book in books if book.get("author")}
```
"""

client = OpenAI(api_key="")


stream = client.chat.completions.create(
    model="gpt-4.1",
    messages=[
        {
            "role": "user",
            "content": question,
        },
    ],
    stream=True,
)

response = ""
display_handle = display(Markdown(""), display_id=True)

for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    
    update_display(Markdown(response), display_id=display_handle.display_id)

## Choices example

In [8]:
from openai import OpenAI

client = OpenAI(api_key="")

completions = client.chat.completions.create(
    model="gpt-4.1",
    messages=[
        {
            "role": "user",
            "content": "Tell me a joke.",
        },
    ],
    n=5
)

for index, choice in enumerate(completions.choices):
    print(f"Choice {index + 1}")
    print(choice.message.content)

Choice 1
Why did the scarecrow win an award?  
Because he was outstanding in his field!
Choice 2
Why did the scarecrow win an award?

Because he was outstanding in his field!
Choice 3
Why did the scarecrow win an award?

Because he was outstanding in his field!
Choice 4
Why don’t skeletons fight each other?  
They don’t have the guts!
Choice 5
Why don’t skeletons fight each other?  
They don’t have the guts!
